In [1]:
#libraries
from itertools import combinations, chain
from  more_itertools import unique_everseen
import time
import random
#import data and preprocess
data = open("adult.csv","r")

#sampling improvement for Apriori
samplingfactor = .6

In [2]:
#input data and each observation as a list within a data list structure
#add _i to data value indicating i'th variable
def clean(dta,samplingfactor):
    datalist = []
    for line in dta:
        linesep = line.split(", ")
        for attribute in linesep:
            newattribute = attribute  + "_" + str(linesep.index(attribute))
            linesep[linesep.index(attribute)] = newattribute
        datalist.append(linesep)
        
    random.shuffle(datalist) #randomizes data order
    datalist = [datalist[i] for i in range(0,int(round(samplingfactor*len(datalist))))] #return the first sampfactor
    return datalist

In [3]:
# obtaining C1 - candidate set with one attribute
def gen_C1(dta):
    count_dict = {}
    for observation in dta:
        for val in observation:
            if val not in count_dict:
                count_dict[val] = 1
            else:
                count_dict[val] += 1
    
    return count_dict

In [4]:
# obtaining L1 - frequent set derived by C1
# Q: what is sampling factor referring to?
def gen_L1(dic, min_supp, samplingfactor):
    L1 = []
    for var in dic:
        if float(dic[var])/float(n_obs)>min_supp*samplingfactor:
            L1.append(var)
    return L1

In [5]:
# check if k-1 subsets of k itemset are all in L_k-1
# if any k-1 subset of itemset is not frequent, then rule the k-itemset out
def has_infreq_subset(k,L,c):
    for i in list(combinations(c,k-1)):
        if i not in L:
            return False
        else:
            return True

In [6]:
# generate C2 from L1
def gen_C2(k,L):
    Ck = list(combinations(L,k))
    for c in Ck:
        if has_infreq_subset(k,L,c):
            Ck.remove(c) # if it has infrequent subset, remove the candidate
        else:
            pass
    return Ck

In [7]:
def gen_Lk(Ck,min_supp,dta,samplingfactor):
    count_dict = {}
    for c in Ck:
        count_dict[c] = 0
        
    for observation in dta:
        for c in Ck:
            if set(c).issubset(observation):
                count_dict[c] +=1
    
    Lk = []
    for c in Ck:
        if float(count_dict[c])/n_obs>min_supp*samplingfactor:
            Lk.append(c)
            
    return Lk

In [8]:
def gen_Ck(k,L):
    flatten = [item for subtuple in L for item in subtuple] # flatten all candidates
    uniqueflatten = list(unique_everseen(flatten)) # remove all duplicate candidates
    Ck = list(combinations(uniqueflatten,k)) # all possible combinations of k length
    for c in Ck:
        if has_infreq_subset(k,L,c):
            Ck.remove(c)
        else:
            pass
    return Ck

In [9]:
def Apriori(k,L,dta,min_supp,samplingfactor):
    l_of_L = [] # store different Lk
    while L!=[]:
        if k==2:
            Ck = gen_C2(k,L)
        else:
            Ck = gen_Ck(k,L)
        L = gen_Lk(Ck,min_supp,cleandata,samplingfactor)
        l_of_L.append(L)
        k+=1
        
    return l_of_L[0:len(l_of_L)-1]

In [10]:
time1 = time.time()
cleandata = clean(data,samplingfactor)
n_obs = len(cleandata)

print('The first few lines of input data are:'+'\n')
print(cleandata[0:3])

C1 = gen_C1(cleandata)
L1 = gen_L1(C1,0.75,samplingfactor)
freqsets = Apriori(2,L1,cleandata,0.6,samplingfactor)

print('The frequent itemsets are:'+'\n')
for i in freqsets:
    for j in i:
        print(j)

time2 = time.time()

testtime = time2-time1
print('\n')
print('The runtime for this algorithm (with a sampling factor=0.6 and min_supp=0.6) is {}'.format(testtime))

The first few lines of input data are:

[['26_0', 'Private_1', '35917_2', 'HS-grad_3', '9_4', 'Married-civ-spouse_5', 'Sales_6', 'Husband_7', 'White_8', 'Male_9', '0_10', '0_11', '40_12', 'United-States_13', '<=50K\n_14'], ['19_0', '?_1', '307837_2', 'Some-college_3', '10_4', 'Never-married_5', '?_6', 'Own-child_7', 'White_8', 'Male_9', '0_10', '0_11', '40_12', 'United-States_13', '<=50K\n_14'], ['26_0', 'Private_1', '101027_2', 'HS-grad_3', '9_4', 'Married-civ-spouse_5', 'Sales_6', 'Husband_7', 'White_8', 'Male_9', '0_10', '0_11', '48_12', 'United-States_13', '<=50K\n_14']]
The frequent itemsets are:

('Private_1', 'White_8')
('Private_1', 'Male_9')
('Private_1', '0_10')
('Private_1', '0_11')
('Private_1', 'United-States_13')
('Private_1', '<=50K\n_14')
('Married-civ-spouse_5', 'White_8')
('Married-civ-spouse_5', 'Male_9')
('Married-civ-spouse_5', '0_10')
('Married-civ-spouse_5', '0_11')
('Married-civ-spouse_5', 'United-States_13')
('White_8', 'Male_9')
('White_8', '0_10')
('White_8',